# Chapter 2 
## Regular Expressions, Text Normalization and Edit Distance

Regular expressions are useful in analysing data. Their use is not limited to only NLP, often we want to extract specific values or patterns from a document. Furthermore this tool is often supplied in many common commandline tools as a means to search the file system and its contents.

Turning to NLP their value centers around tasks we will call **text normalization** and **tokenization**. **Text normalization** as its name implies refers to tasks or tricks used by nlp practitioners to normalize written text into some standard form. This can be not only things such as fixing spelling errors, correcting numbers to all use the same formatting, and making all nouns singular but also **sentence segmentation** (splitting text to sentences).

**Tokenization** the task of splitting words is intuitive to humans, however describing the exceptions, or errors which appear in real world data in terms of a regular expression are difficult. Examples of such exceptions are the following:

    - New York
    - rock'n'roll
    - emoticons: ':)'
    - hashtags: #blm
    
As shown in the text, regular expressions can also be used to parse text in a predictable format and extract subjects, however these types of applications are very rigid and labor intensive.

Another task in **text normalization** is **lemmatization** and it's simplified counterpart **stemming**. **Lemmatization** is the process of mapping words to their roots, for example *sings*, *sang*, and *sung* all share the root *sing*. While english is one of the most morphologically simplest languages, many other languages allow words to adapt to not only tense, but gender, person, etc. making the task of mapping a word to its **lemma** much more labor intensive. To this end, **stemming** is a pragmatic approach which removes suffixes from words, in an attempt to strip them to some common subword.